# Book Recommendation Hackathon 

**Task:** Rank 20 editions for each user from 200 candidates, optimizing Score = 0.7×NDCG@20 + 0.3×Diversity@20

**Strategy - classic, catboost ranker + rearranging (for the 30% of the residual metric bcs catboost is fitted on ndcg)**

In [4]:
import sys 
import os
import warnings 

os.environ['OPENBLUS_NUM_THREADS'] = '1'
warnings.filterwarnings('ignore')

In [5]:
import pandas as pd
import datetime as dt
import numpy as np

interactions = pd.read_csv('data/interactions.csv')
editions = pd.read_csv('data/editions.csv')
users = pd.read_csv('data/users.csv')
book_genres = pd.read_csv('data/book_genres.csv')
genres = pd.read_csv('data/genres.csv')
authors = pd.read_csv('data/authors.csv') 
target_users = pd.read_csv('submit/targets.csv') 
target_interactions = pd.read_csv('submit/candidates.csv')

print('all data frames have been loaded successfully')

all data frames have been loaded successfully


In [6]:
%%time

interactions['event_ts'] = pd.to_datetime(interactions['event_ts'])

split_date = pd.Timestamp('2025-03-12')

feature_source = interactions.loc[interactions['event_ts'] < split_date]
train = interactions.loc[interactions['event_ts'] > split_date]

CPU times: user 37 ms, sys: 12.8 ms, total: 49.8 ms
Wall time: 61.3 ms


In [7]:
%%time

book_genres = book_genres.groupby('book_id')['genre_id'].apply(lambda x: ' '.join(x.astype(str))).reset_index()
enriched_editions = editions.merge(book_genres, on='book_id')

enriched_editions['author_productivity']= enriched_editions.author_id.map(enriched_editions.author_id.value_counts())

feature_source = feature_source.drop('event_ts', axis=1)
feature_source = feature_source.merge(users, on='user_id')
feature_source = feature_source.merge(enriched_editions, on='edition_id')

feature_source = feature_source.drop('book_id', axis=1) #1 to 1 with edition_id
feature_source = feature_source.drop('publisher_id', axis=1) #1 to 1 with edition_id

feature_source['edition_popularity_score'] = feature_source.edition_id.map(feature_source.edition_id.value_counts())
feature_source['reader_mean_age'] = feature_source.groupby('edition_id')['age'].transform('mean')
feature_source['book_age'] = 2026 - feature_source['publication_year']
feature_source['user_mean_rating'] = feature_source.groupby('user_id')['rating'].transform('mean')
feature_source['book_mean_rating'] = feature_source.groupby('edition_id')['rating'].transform('mean')
feature_source = feature_source.drop('rating', axis=1)

user_cols = ['user_id', 'gender', 'age', 'user_mean_rating']

user_features = feature_source[user_cols].drop_duplicates().reset_index()
user_features = user_features.drop('index', axis=1)

book_features = feature_source[[f for f in feature_source.columns.to_list() if f not in user_cols]].drop_duplicates().reset_index()
book_features = book_features.drop(['event_type', 'index'], axis=1)
book_features = book_features.drop_duplicates()

train = train.merge(book_features, on='edition_id').merge(user_features, on='user_id')
train = train.drop(['event_ts', 'rating'], axis=1)

CPU times: user 1.96 s, sys: 199 ms, total: 2.16 s
Wall time: 2.19 s


In [8]:
train.head(1)

,user_id,edition_id,event_type,author_id,publication_year,age_restriction,language_id,title,description,genre_id,author_productivity,edition_popularity_score,reader_mean_age,book_age,book_mean_rating,gender,age,user_mean_rating
0,560,1010822636,2,507926.0,2024,16,119,Призраки белых ночей,На встрече бывших однокурсников Александра ста...,1222 1224 1309,56,7,37.857143,2,7.571429,2.0,9.0,7.733333


In [10]:
%%time

import numpy as np
import pandas as pd

# 1. Настройки
NEGATIVE_FRACTION = 3
needed_cols = [
    'user_id', 'edition_id', 'event_type', 'gender', 'age', 'author_id', 
    'publication_year', 'age_restriction', 'language_id', 'title', 
    'description', 'genre_id', 'author_productivity', 
    'edition_popularity_score', 'reader_mean_age', 'book_age', 
    'user_mean_rating', 'book_mean_rating'
]

# 2. Подготовка количества сэмплов для каждого юзера
# Вычисляем, сколько негативных примеров нужно каждому пользователю
num_negative_samples_per_user = train['user_id'].value_counts() * NEGATIVE_FRACTION
max_books = len(book_features)
num_negative_samples_per_user = num_negative_samples_per_user.clip(upper=max_books)

# 3. Подготовка признаков пользователей (для быстрого джойна)
_user_cols = ['user_id', 'gender', 'age', 'user_mean_rating']
_user_features = train[_user_cols].drop_duplicates(subset=['user_id']).set_index('user_id')

# 4. Генерация массивов для сэмплирования
user_ids = num_negative_samples_per_user.index.to_numpy()
counts = num_negative_samples_per_user.to_numpy()

# Создаем длинный массив user_id, где каждый ID повторяется N раз
repeated_user_ids = np.repeat(user_ids, counts)
total_samples = len(repeated_user_ids)

# 5. СЛУЧАЙНЫЙ ВЫБОР КНИГ (вместо сортировки по популярности)
# Генерируем случайные индексы книг
random_indices = np.random.randint(0, max_books, size=total_samples)

# Выбираем книги по этим случайным индексам
# reset_index нужен, чтобы iloc работал корректно с 0 до max_books
sampled_books = book_features.reset_index(drop=True).iloc[random_indices].reset_index(drop=True)

# 6. Подтягиваем данные пользователей
sampled_users = _user_features.loc[repeated_user_ids].reset_index(drop=True)

# 7. Сборка датафрейма с негативными примерами
negativity_builder = pd.concat([sampled_users, sampled_books], axis=1)
negativity_builder['user_id'] = repeated_user_ids
negativity_builder['event_type'] = 0  # Маркируем как негативный пример
negativity_builder = negativity_builder[needed_cols]

# 8. Объединение с основным трейном и чистка
train = pd.concat([train, negativity_builder], ignore_index=True)

# Сортируем так, чтобы реальные события (event_type != 0) были выше
train = train.sort_values(by='event_type', ascending=False)

# Удаляем дубликаты.
# Если случайная книга оказалась той, которую юзер реально читал,
# благодаря сортировке мы оставим верхнюю запись (реальную), а нижнюю (фейковую) удалим.
train = train.drop_duplicates(subset=['user_id', 'edition_id'], keep='first')

# Финальное форматирование
train.edition_id = train.edition_id.astype(str)
train = train.reset_index(drop=True)

CPU times: user 66.2 ms, sys: 107 ms, total: 173 ms
Wall time: 194 ms


In [11]:
train.head(3)

,user_id,edition_id,event_type,author_id,publication_year,age_restriction,language_id,title,description,genre_id,author_productivity,edition_popularity_score,reader_mean_age,book_age,book_mean_rating,gender,age,user_mean_rating
0,560,1010822636,2,507926.0,2024,16,119,Призраки белых ночей,На встрече бывших однокурсников Александра ста...,1222 1224 1309,56,7,37.857143,2,7.571429,2.0,9.0,7.733333
1,7163500,1007326489,2,2011018.0,2022,18,0,Клара и 11 бабушек,Кларе 9 лет и она живет с дедушкой-трубочистом...,137,1,1,31.000000,4,NaN,2.0,11.0,9.628571
2,7168360,1012387985,2,1199919.0,2025,18,119,Мальчики в долине,"Конец XIX века, горная долина в штате Пенсильв...",1223,2,10,35.333333,1,6.000000,2.0,28.0,7.347826


In [17]:
from catboost import CatBoostRanker, Pool
from sklearn.model_selection import train_test_split

cat_features = [
    'gender', 
    'author_id', 
]

text_features = [
    'title', 
    'description', 
    'genre_id'
]
# Сбрасываем индекс СРАЗУ, чтобы он был 0, 1, 2... везде
train = train.sort_values('user_id').reset_index(drop=True)

# 2. Выделяем группы (Queries) и таргет (Label) ПЕРЕД удалением колонок
queries = train['user_id']
label = train['event_type']

# 3. Готовим признаки (X) - удаляем ВСЁ лишнее тут
# ВАЖНО: Убираем user_id отсюда, чтобы не было лика!
drop_cols = ['event_type', 'edition_id', 'user_id', 
             'edition_popularity_score', 'user_mean_rating', 
             'book_mean_rating', 'reader_mean_age']          
X = train.drop(columns=drop_cols)

# 4. Приводим категории к строкам
cat_features_safe = ['author_id', 'language_id', 'gender', 'genre_id'] 
for col in cat_features_safe:
    X[col] = X[col].astype(str).replace('nan', 'unknown')

# 5. МАГИЧЕСКИЙ СПЛИТ: Передаем X, y и queries ВМЕСТЕ
# shuffle=False обязателен для ранкера, чтобы не разорвать группы юзеров
X_train, X_test, y_train, y_test, q_train, q_test = train_test_split(
    X, label, queries, 
    test_size=0.33, 
    random_state=42, 
    shuffle=False 
)

# 6. Проверка на вшивость (Обязательно посмотри этот принт!)
print(f"Размер групп в трейне: {q_train.value_counts().mean():.2f} книг на юзера")
# Если тут будет 1.0 — значит данные кривые. Должно быть 5-10-20.

# Заполни пустоты во ВСЕХ текстовых признаках
text_cols = ['title', 'description', 'genre_id'] # проверь список по своему коду

for col in text_cols:
    X_train[col] = X_train[col].fillna('none').astype(str)
    X_test[col] = X_test[col].fillna('none').astype(str)

# Теперь создавай Pool
train_pool = Pool(
    data=X_train,
    label=y_train,
    group_id=q_train,
    cat_features=cat_features,
    text_features=text_features
)

val_pool = Pool(
    data=X_test, label=y_test, group_id=q_test,
    cat_features=cat_features_safe, text_features=text_cols
)

Размер групп в трейне: 26.79 книг на юзера


In [89]:
%%time

from catboost import CatBoostRanker

TASK_TYPE = 'CPU'

model = CatBoostRanker(
    iterations=1000, 
    learning_rate=0.1, 
    loss_function='YetiRank', 
    eval_metric='NDCG:top=20', 
    random_seed=42, 
    task_type=TASK_TYPE, 
    metric_period=100, 
    use_best_model=True, 
    early_stopping_rounds=100
)

model.fit(
    train_pool,
    eval_set=val_pool, 
    plot=True
)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Groupwise loss function. OneHotMaxSize set to 10


0:	test: 0.2957117	best: 0.2957117 (0)	total: 230ms	remaining: 3m 49s
100:	test: 0.2719063	best: 0.2957117 (0)	total: 19.1s	remaining: 2m 49s
Stopped by overfitting detector  (100 iterations wait)

bestTest = 0.2957117316
bestIteration = 0

Shrink model to first 1 iterations.
CPU times: user 2min 4s, sys: 5.19 s, total: 2min 9s
Wall time: 21 s


In [79]:
# 1. Проверяем, что модель обучена. Если выведет True, значит всё ок.
print(f"Is model trained: {model.is_fitted()}")

# 2. Явно запрашиваем важность
importances = model.get_feature_importance(train_pool)
feature_names = model.feature_names_

# 3. Собираем таблицу
fea_imp = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values(by='importance', ascending=False)

print(fea_imp)

Is model trained: True
                feature  importance
5           description    0.016096
4                 title    0.007363
10                  age    0.002044
7   author_productivity    0.001471
0             author_id    0.001300
3           language_id    0.000212
2       age_restriction    0.000159
9                gender   -0.000020
1      publication_year   -0.000204
8              book_age   -0.000240
6              genre_id   -0.009451
